In [41]:
!pip install tiktoken
!pip install openai
!pip install PyPDF
!pip install docxtpl

In [42]:
import PyPDF2
from google.colab import files
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text
file = "RFQ-230203-Engineering-Services-for-USDA-Grant.pdf" #file path
pdf_text = extract_text_from_pdf(file)
#getting text from pdf


In [44]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

tokens = encoding.encode(pdf_text)
token_count = len(tokens)
print(token_count)

if token_count>128000: #token counter incase document is too big
  print("please give a smaller pdf")

29181


In [45]:

from openai import OpenAI

client = OpenAI(api_key = api_key)

# prompt = '''you have to extract the templete such that later on I can generate a similar document using docxtpl with same structure but different body content


# '''
prompt =  '''you will be given a text you have to extract a templete from that and return the templete in a json ojbect, only return the empty templete
rules you must follow
1. do not leave any heading or subheading from the text
2.only remove common body which can be changed in the document
3.keep the document format same
4. extract the templete such that exact same type of document can be generated using using the output json
5. do not make nested json object keep the json easy to parse
6. start object which would have larger font size with +
7. only keep a single dict in json so it can be easily parsed

heres an example

{
  "+title": "title here",
  "date": "{{date}}"
   "name": "{{name}}"
  "checklist":"{{checklist}}"
  "+abstract":"{{+abstract}}"
  "+introduction":"{{+introduction}}"
  "+methodology":"{{+methodology}}"
  "+results":"{{+results}}"
  "+discussion":"{{+discussion}}"
  "+conclusion":"{{+conclusion}}
  "+any other heading or sub heading":"{+any other heading or sub heading}}",
}"
 "'''
data = """"""
message = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": pdf_text}
]
response = client.chat.completions.create(
  model="gpt-4o",
  temperature=0,
  messages=message,
  response_format = { "type": "json_object" }
)

jsonformat = response.choices[0].message.content
print(jsonformat)

{
  "+title": "CAMERON COUNTY PURCHASING DEPARTMENT REQUEST FOR QUALIFICATIONS",
  "RFQ Number": "{{RFQ Number}}",
  "RFQ Title": "{{RFQ Title}}",
  "Date Due": "{{Date Due}}",
  "Opening Location": "{{Opening Location}}",
  "Return RFQ To": "{{Return RFQ To}}",
  "Contact Information": "{{Contact Information}}",
  "Company Name": "{{Company Name}}",
  "Company Address": "{{Company Address}}",
  "City, State, Zip Code": "{{City, State, Zip Code}}",
  "Historically Underutilized Business Certification VID Number": "{{Historically Underutilized Business Certification VID Number}}",
  "Telephone No": "{{Telephone No}}",
  "Fax No": "{{Fax No}}",
  "Email": "{{Email}}",
  "Print Name": "{{Print Name}}",
  "Signature": "{{Signature}}",
  "How did you find out about this RFQ?": "{{How did you find out about this RFQ?}}",
  "Is Bidder’s principal place of Business within Cameron County?": "{{Is Bidder’s principal place of Business within Cameron County?}}",
  "If yes what City": "{{If yes wha

In [46]:
from docx import Document
from docx.shared import Pt
import json

# Define your JSON-like structure
json_data = json.loads(jsonformat)

# Create a new Document
doc = Document()

# Function to add a section with formatting
def add_section(doc, title, content, is_large_font=False):
    para = doc.add_paragraph()
    run = para.add_run(title + ': ' + content)
    if is_large_font:
        run.font.size = Pt(14)  # Larger font size for sections starting with #
    else:
        run.font.size = Pt(10)  # Regular font size

# Add the title
# doc.add_heading(json_data.get("#title", ""), level=1)

# Iterate over JSON data and add sections to the document
for key, value in json_data.items():
    if key.startswith('+'):
        add_section(doc, key[1:], value, is_large_font=True)
    else:
        add_section(doc, key, value)

# Save the document
doc.save('template.docx')

print("Document created successfully.")

Document created successfully.


In [47]:
user_prompt = "generate dumy data"#user prompt

In [48]:
def gpt(user):
  message.append({"role": "user", "content": f"field = {user}"})
  response = client.chat.completions.create(
  model="gpt-4o",
  temperature=1,
  messages=message
  )
  # print(response.choices[0].message.content)
  return response.choices[0].message.content




prompt = f'''forget the pervious prompt or instructions now your job is to fill the requested field according to user prompt only return text which is to be filed in that field
following these rule while generating text
1. only give the content body nothing else
2. do not give a heading or name the requested field as it would be pre field
3. follow user prompt
user prompt = {user_prompt}
'''
message = []
message.append({"role": "system", "content": prompt})

fields = {}
for key, value in json_data.items():
    fields[f"{value}"] = f"{gpt(value)}"

print(fields)

{'CAMERON COUNTY PURCHASING DEPARTMENT REQUEST FOR QUALIFICATIONS': 'This is a request for qualifications for potential contractors for upcoming projects within Cameron County. Interested parties should provide credentials, previous project experiences, and pertinent qualifications for consideration.', '{{RFQ Number}}': 'RFQ-2023-123', '{{RFQ Title}}': '', '{{Date Due}}': 'September 30, 2023', '{{Opening Location}}': '3001', '{{Return RFQ To}}': '', '{{Contact Information}}': '1234', '{{Company Name}}': 'CCPD-RFQ-2023-007', '{{Company Address}}': '1234 Main Street, Suite 101, Brownsville, TX, 78520', '{{City, State, Zip Code}}': '1234', '{{Historically Underutilized Business Certification VID Number}}': '12345', '{{Telephone No}}': 'CAMERON COUNTY PURCHASING DEPARTMENT REQUEST FOR QUALIFICATIONS\n\n001-2023\n\nArchitectural and Engineering Services\n\nNovember 17, 2023\n\nCameron County Administration Building, Room 105\n\nCameron County Purchasing Department, 1100 E. Monroe St., Brown

In [49]:
from docx import Document
def replaceword(temp, output, data):
  doc = Document(temp)
  for para in doc.paragraphs:
    for key, value in data.items():
      if key in para.text:
        for run in para.runs:
          run.text = run.text.replace(key, value)
  doc.save(output)

data = fields

temp = 'template.docx'
output = 'output.docx'
replaceword(temp, output, data)